In [1]:
import numpy as np
import re
import subprocess
from astropy.table import Table, vstack

In [2]:
#Read the full log. 
log = Table.read("../../proc/log.fits")

In [3]:
#Separate between Hot DOGs and calibrators, as we want first all Hot DOGs and then the calibrators.
hds_cond = ~(log['Program_ID']=="60.A-9203(E)")
ipol_cond  = np.zeros(len(log), dtype=bool)
zipol_cond = np.zeros(len(log), dtype=bool)
for i, ob_name in enumerate(log['OB_Name']):
    if ob_name[:4]=="Zero":
        zipol_cond[i] = True
    if ob_name[:4]=="IPOL":
        ipol_cond[i] = True
hds        = log[hds_cond]
ipol_cals  = log[ipol_cond]
zipol_cals = log[zipol_cond]
cals = vstack([ipol_cals, zipol_cals])

In [4]:
#Put the target names in order. First Hot DOGs, then Cal Stars and then Zero Pol stars. As all zero pol stars have names starting WD, cals is already arange in the proper way. 
targs = np.concatenate([np.unique(hds['Target'].astype(str)), np.unique(cals['Target'].astype(str))])

In [5]:
#Main cycle. 
aux = open("aux.txt","w")
aux.write("Target,Band,OB,pfrac,epfrac,chi,echi\n")
for i, targ in enumerate(targs):
    for j, filt in enumerate(np.unique(log['Filter'][log['Target']==targ])):
        pcat_name = "../../analysis_v2/{}_pol_{}.dat".format(targ,filt)
        try:
            cat = open(pcat_name)
        except FileNotFoundError:
            continue
        for k, line in enumerate(cat.readlines()):
            line = re.sub("All","Combined",line)
            x = line.split()
            if len(x)>9:
                continue
            x[:7] = [float(ix) for ix in x[:7]]
            row =  [targ, filt]+[x[i] for i in [7,0,1,2,3]]
            for ir, r in enumerate(row):
                if ir>0:
                    aux.write(",")
                aux.write("{}".format(r))
            aux.write("\n")
aux.close()


In [6]:
tab = Table.read("aux.txt",format='ascii.csv')
subprocess.call("rm aux.txt",shell=True)

0

In [7]:
#We will create two ne columns that show measurement and error for pol frac and angle. 
tab['P'] = np.zeros(len(tab), dtype='<U18')
tab['\chi'] = np.zeros(len(tab), '<U18')
for i in range(len(tab)):
    tab['P'][i] = "{:.2f}$\\pm${:.2f}".format(tab['pfrac'][i], tab['epfrac'][i])
    tab['\chi'][i] = "{:.1f}$\\pm${:.1f}".format(tab['chi'][i], tab['echi'][i])
tab['P'].unit = "(%)"
tab['\chi'].unit = "(deg)"

In [8]:
#Remove unneeded columns.
for col in ['pfrac', 'epfrac', 'chi', 'echi']:
    tab.remove_column(col)

In [9]:
filt_name_dict = {
    "R_SPECIAL": "$R_{\\rm Special}$",
    "I_BESS": "$I_{\\rm Bessel}$",
    "v_HIGH": "$v_{\\rm High}$"
}

def add_lines(tab, cato):
    for i in range(len(tab)):
        #Indent
        cato.write("{:8s}".format(""))

        #Target
        targ_name = ""
        if i==0 or tab['Target'][i]!=tab['Target'][i-1]:
            targ_name = re.sub("-","--",tab['Target'][i])
        cato.write("{:12s} &".format(targ_name))

        #Filter
        filt_name = ""
        if i==0 or tab['Target'][i]!=tab['Target'][i-1] or tab['Band'][i]!=tab['Band'][i-1]:
            filt_name = filt_name_dict[tab['Band'][i]]
        cato.write(" {:17s} &".format(filt_name))

        #OB
        cato.write(" {:9} &".format(tab['OB'][i]))

        #Pol frac and Error
        pol_val = tab['P'][i]
        if float(tab['P'][i].split("$")[0])<10.:
            pol_val = "\phantom{0}"+pol_val
        cato.write(" {:>24} &".format(pol_val))

        #Pol angle and Error
        pang_val = tab['\chi'][i]
        if float(tab['\chi'][i].split("$")[0])<100.:
            pang_val = "\phantom{0}"+pang_val
        cato.write(" {:>23} \\\\".format(pang_val))

        #End the line.
        cato.write("\n")
    return

In [10]:
#Now, go and edit the table. 
temp = open("template.tex")
cato = open("pol_table.tex","w")
for line in temp:
    if re.search("INSERT_",line):
        x = line.split()
        if x[0][7:]=="BHDS":
            aux_tab = tab[np.isin(tab['Target'], hds['Target'])]
        elif x[0][7:]=="POLSTDS":
            aux_tab = tab[np.isin(tab['Target'], ipol_cals['Target'])]
        elif x[0][7:]=="ZPOLSTDS":
            aux_tab = tab[np.isin(tab['Target'], zipol_cals['Target'])]
        else:
            print("Wrong object type in template.")
            continue
        add_lines(aux_tab, cato)
        continue
    cato.write(line)
cato.close()
